# 📘 Notebook 7 – Merging Transaction & Survey Personas

**Goal:**  
To create a unified investor profile by combining:
- Transaction-based personas (objective behaviors)
- Survey-based personas (subjective attitudes & preferences)

**Why this matters:**  
Transaction data tells us *what investors do*, while survey data tells us *why they do it*.  
By merging these perspectives, we can build richer investor archetypes for PortfoliAI.

**Steps in this notebook:**
1. Load transaction personas (Notebook 6 output) & survey responses (`investors.csv`)  
2. Clean & encode survey data into usable features/personas  
3. Merge both datasets on user identifiers  
4. Generate a combined investor profile for each user  


In [2]:
# --- Cell 1: Load Datasets ---

from google.colab import drive
import os
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

# Define base path (adjust if needed)
base_path = "/content/drive/My Drive/portfoliai/data/processed"

# Now load transaction personas
trans_path = os.path.join(base_path, "transaction_personas.csv")
transaction_personas = pd.read_csv(trans_path)

print(f"✅ Loaded {transaction_personas.shape[0]} rows from transaction_personas.csv")

Mounted at /content/drive
✅ Loaded 32124 rows from transaction_personas.csv


In [4]:
# --- Cell 2: Load survey (investors.csv) and transaction_personas (if present) ---

import os
import pandas as pd
from IPython.display import display

# Paths (assumes you mounted Google Drive in Cell 1)
RAW_DIR = "/content/drive/My Drive/portfoliai/data/raw"
PROC_DIR = "/content/drive/My Drive/portfoliai/data/processed"

survey_path = os.path.join(RAW_DIR, "investors.csv")
trans_path  = os.path.join(PROC_DIR, "transaction_personas.csv")

# 1) Load survey (investors.csv) — required
if os.path.exists(survey_path):
    survey_df = pd.read_csv(survey_path)
    print(f"✅ Loaded survey file: {survey_path}")
    print(f"   rows: {survey_df.shape[0]}, cols: {survey_df.shape[1]}")
else:
    raise FileNotFoundError(f"investors.csv not found at: {survey_path}. Please upload the survey CSV to /data/raw.")

# 2) Try to load transaction_personas (optional)
if os.path.exists(trans_path):
    transaction_personas = pd.read_csv(trans_path)
    print(f"✅ Loaded transaction_personas: {trans_path}")
    print(f"   rows: {transaction_personas.shape[0]}, cols: {transaction_personas.shape[1]}")
else:
    transaction_personas = None
    print(f"⚠️ transaction_personas.csv not found at: {trans_path}. Continuing with survey-only workflow for now.")

# 3) Quick preview of the survey and transaction (if present)
print("\n--- Survey (investors.csv) preview ---")
display(survey_df.head(6))
print("\nColumns in survey_df:")
for c in survey_df.columns:
    print("  -", c)

if transaction_personas is not None:
    print("\n--- transaction_personas preview ---")
    display(transaction_personas.head(6))
    print("\nColumns in transaction_personas:")
    for c in transaction_personas.columns:
        print("  -", c)

# Keep variables survey_df and transaction_personas available for the next cells


✅ Loaded survey file: /content/drive/My Drive/portfoliai/data/raw/investors.csv
   rows: 132, cols: 29
✅ Loaded transaction_personas: /content/drive/My Drive/portfoliai/data/processed/transaction_personas.csv
   rows: 32124, cols: 4

--- Survey (investors.csv) preview ---


,Timestamp,Username,Which best describes your gender?,Age,What is your occupation?,What is your highest education level?,Do you invest in Investment Avenues?,What proportion of money you invest?,What do you think are the best options for investing your money? (Rank in order of preference) [Mutual Funds],What do you think are the best options for investing your money? (Rank in order of preference) [Equity Market],...,How often do you monitor your investments?,How much return do you expect from any investment instrument?,Which investment avenue do you mostly invest in?,What are your savings objectives?,Reasons for investing in Equity Market,Reasons for investing in Mutual Funds,What is your purpose behind investment?,Reasons for investing in Government Bonds,Reasons for investing in Fixed Deposits,Your major source of information for investment is
0,2023/04/09 9:14:59 PM GMT+5:30,NaN,Male,39,Salaried,Post Graduate,Yes,10% - 20%,3,3,...,Monthly,10% - 20%,Fixed Deposits,Retirement Plan,Power of compounding,Better Returns,Savings for retirement,Safe Investment,Fixed returns,Newspapers and Magazines
1,2023/04/09 9:25:23 PM GMT+5:30,NaN,Male,30,Salaried,Post Graduate,Yes,20% - 30%,3,6,...,Weekly,10% - 20%,Equity,Retirement Plan,Capital Appreciation,Better Returns,Wealth creation,Risk-free returns,Fixed returns,Internet
2,2023/04/09 9:34:18 PM GMT+5:30,NaN,Female,26,Salaried,Professional Degree,No,30% and above,7,4,...,Weekly,10% - 20%,Mutual Funds,Retirement Plan,Liquidity,Better Returns,Wealth creation,Risk-free returns,Fixed returns,Internet
3,2023/04/09 9:36:55 PM GMT+5:30,NaN,Male,32,Salaried,Post Graduate,Yes,10% - 20%,7,5,...,Monthly,10% - 20%,Mutual Funds,Retirement Plan,Capital Appreciation,Funds Diversification,Savings for retirement,Risk-free returns,Risk free,Internet
4,2023/04/09 9:37:42 PM GMT+5:30,NaN,Male,28,Salaried,Professional Degree,Yes,30% and above,4,6,...,Daily,10% - 20%,Equity,Retirement Plan,Capital Appreciation,Tax Benefits,Wealth creation,Tax Incentives,Fixed returns,Internet
5,2023/04/09 9:54:42 PM GMT+5:30,NaN,Male,35,Salaried,Post Graduate,Yes,20% - 30%,3,1,...,Monthly,10% - 20%,Equity,Retirement Plan,Capital Appreciation,Better Returns,Wealth creation,Tax Incentives,High Interest Rates,Newspapers and Magazines



Columns in survey_df:
  - Timestamp
  - Username
  - Which best describes your gender?
  - Age
  - What is your occupation?
  - What is your highest education level?
  - Do you invest in Investment Avenues?
  - What proportion of money you invest?
  - What do you think are the best options for investing your money? (Rank in order of preference) [Mutual Funds]
  - What do you think are the best options for investing your money? (Rank in order of preference) [Equity Market]
  - What do you think are the best options for investing your money? (Rank in order of preference) [Corporate Bonds]
  - What do you think are the best options for investing your money? (Rank in order of preference) [Government Bonds]
  - What do you think are the best options for investing your money? (Rank in order of preference) [Fixed Deposits]
  - What do you think are the best options for investing your money? (Rank in order of preference) [PPF - Public Provident Fund]
  - What do you think are the best options

,customerID,risk_score_scaled,abs_persona,pct_persona
0,00017496858921195E5A,0.061900,Aggressive,Risk Seeker
1,00017496858921195E5A,0.061900,Aggressive,Risk Seeker
2,00017496858921195E5A,0.061900,Aggressive,Risk Seeker
3,00017496858921195E5A,0.061900,Aggressive,Risk Seeker
4,00024864C985E72167A0,0.000183,Conservative,Steady Saver
5,0004718496C71D464F57,0.014933,Balanced,Risk Seeker



Columns in transaction_personas:
  - customerID
  - risk_score_scaled
  - abs_persona
  - pct_persona


What this step does

Loads your survey data: investors.csv from portfoliai/data/raw. This is the single source of survey responses you told me you have (100 rows in your sample). We treat this as the "survey_df" for the rest of the notebook.

Attempts to load transaction-derived personas: transaction_personas.csv from portfoliai/data/processed. If that file is present (it might have been created in Notebook 6 when we processed transactions), we load it; if not, we continue with a survey-only workflow — no error, just a warning.

Shows a quick preview and column list so you can visually confirm the data and column names.

Why we do this now

We need the survey (investors.csv) to build the survey → personality / risk mapping. That model maps survey responses (answers, rankings, time-horizon, monitoring frequency, expected return, etc.) to a risk/persona output.

The transaction_personas file (if present) is valuable because it contains transaction-based risk/persona signals (the outputs from Notebook 4–6). When available we’ll merge the two datasets (survey + transaction personas) so we can:

(a) Label survey respondents using transaction-derived risk/persona (so we can train a model mapping survey → transaction-based risk), or

(b) use both sources together in a hybrid inference engine (survey features and behavioral transaction features).

If transaction_personas.csv is missing, we can still:

Train a survey-only model (labels can be pseudo-labels from earlier models if you have them saved), or

Use the survey model for onboarding users until we collect sufficient transaction data to improve/merge later.

What to expect next (Cell 3 plan)

Clean and transform the survey columns (turn rank answers, frequency, time horizons and text categories into numeric / categorical features).

Build a small feature pipeline that extracts a few interpretable trait scores (e.g., short-term focus, expected-return preference, monitoring frequency → proxy for impulsivity/trading tendency).

If transaction_personas exists: merge on a key (see below). If not: create survey_personas.csv (processed version) for notebook 7 to use.

Important note about joining the datasets

To merge survey records with transaction personas we need a common key (e.g., customerID, Username, email). Your sample shows a Timestamp and maybe Username column — check the displayed columns for a unique identifier. If there is no shared identifier, merging will only work for users who have both records and an agreed key; otherwise we proceed with survey-only modelling until mapping keys are available.

This cell below prepares the survey for modeling. We will:

automatically locate relevant columns from investors.csv (gender, age, invest proportion, monitoring frequency, expected returns, ranks for asset types, etc.)

map categorical answers to numeric proxies (e.g., "20% - 30%" → 0.25)

produce one-hot encodings for categorical choices where useful

save the processed feature table for later steps

In [5]:
# === Cell 3: Preprocess survey_df into numeric features (survey_features.csv) ===

import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Paths (assumes drive already mounted and RAW_DIR/PROC_DIR from Cell 2 exist)
RAW_DIR = "/content/drive/My Drive/portfoliai/data/raw"
PROC_DIR = "/content/drive/My Drive/portfoliai/data/processed"
os.makedirs(PROC_DIR, exist_ok=True)

# Use survey_df already loaded in previous cell
df = survey_df.copy()

# Utility: find the first column name that contains substring (case-insensitive)
def find_col(df, substr):
    substr = substr.lower()
    for c in df.columns:
        if substr in c.lower():
            return c
    return None

# Identify columns robustly
col_gender = find_col(df, "gender")
col_age = find_col(df, "age")
col_occupation = find_col(df, "occupation")
col_education = find_col(df, "education")
col_invest_yes = find_col(df, "do you invest")
col_invest_prop = find_col(df, "proportion of money")
col_monitor = find_col(df, "how often do you monitor")
col_return = find_col(df, "how much return")
col_horizon = find_col(df, "how long do you prefer")
col_stock_pref = find_col(df, "equity market")  # ranking column for equity
col_mutual_pref = find_col(df, "mutual funds")  # ranking column for mutual funds
col_invest_stock_bool = find_col(df, "do you invest in stock market")
col_main_avenue = find_col(df, "which investment avenue do you mostly invest in")

# Create feature DataFrame
feat = pd.DataFrame(index=df.index)

# Gender -> numeric (Male=1, Female=0, else NaN)
if col_gender:
    feat['gender_male'] = df[col_gender].str.lower().eq('male').astype(float)
else:
    feat['gender_male'] = np.nan

# Age -> numeric
if col_age:
    feat['age'] = pd.to_numeric(df[col_age], errors='coerce')
else:
    feat['age'] = np.nan

# Occupation -> keep as category (one-hot later)
if col_occupation:
    feat['occupation'] = df[col_occupation].fillna('Other').astype(str)
else:
    feat['occupation'] = 'Unknown'

# Education -> map to ordinal roughly
edu_map = {
    "ph. d": 5,
    "phd": 5,
    "professional degree": 4,
    "post graduate": 4,
    "postgraduate": 4,
    "graduate": 3,
    "bachelor": 3,
    "high school": 1,
    "others": 0
}
if col_education:
    feat['education_level'] = df[col_education].astype(str).str.lower().map(lambda x: edu_map.get(x, 2))
else:
    feat['education_level'] = 2

# Invest (Y/N)
if col_invest_yes:
    feat['invests'] = df[col_invest_yes].astype(str).str.lower().map(lambda x: 1 if 'yes' in x else 0)
else:
    feat['invests'] = 0

# Invest proportion -> convert ranges to numeric midpoint
prop_map = {
    "5% - 10%": 0.075,
    "10% - 20%": 0.15,
    "20% - 30%": 0.25,
    "30% and above": 0.4,
    "less than 5%": 0.025
}
if col_invest_prop:
    feat['invest_prop_pct'] = df[col_invest_prop].astype(str).map(lambda x: prop_map.get(x.strip(), np.nan))
else:
    feat['invest_prop_pct'] = np.nan

# Monitoring frequency -> map to numeric counts per month (approx)
freq_map = {
    "daily": 30,
    "weekly": 4,
    "monthly": 1,
    "quarterly": 0.3333,
    "yearly": 0.0833,
    "daily ": 30
}
if col_monitor:
    feat['monitor_freq_per_month'] = df[col_monitor].astype(str).str.lower().map(lambda x: freq_map.get(x.strip(), 1))
else:
    feat['monitor_freq_per_month'] = np.nan

# Expected return -> map ranges to midpoint percentages
if col_return:
    feat['expected_return_pct'] = df[col_return].astype(str).map(lambda x: prop_map.get(x.strip(), np.nan))
else:
    feat['expected_return_pct'] = np.nan

# Time horizon -> map to years
horizon_map = {
    "more than 5 years": 7,
    "3-5 years": 4,
    "1-3 years": 2,
    "less than 1 year": 0.5,
    "daily": 0.0
}
if col_horizon:
    feat['horizon_years'] = df[col_horizon].astype(str).str.lower().map(lambda x: horizon_map.get(x.strip(), np.nan))
else:
    feat['horizon_years'] = np.nan

# Preference/rank columns for Equity / Mutual Funds: try to convert to numeric directly
def safe_numeric_col(col):
    if col and col in df.columns:
        return pd.to_numeric(df[col], errors='coerce')
    return pd.Series(np.nan, index=df.index)

feat['rank_equity'] = safe_numeric_col(col_stock_pref)
feat['rank_mutuals'] = safe_numeric_col(col_mutual_pref)

# Do you invest in stock market? -> bool
if col_invest_stock_bool:
    feat['invests_stock'] = df[col_invest_stock_bool].astype(str).str.lower().map(lambda x: 1 if 'yes' in x else 0)
else:
    feat['invests_stock'] = 0

# Main avenue -> one-hot later
if col_main_avenue:
    feat['main_avenue'] = df[col_main_avenue].astype(str).fillna('Unknown')
else:
    feat['main_avenue'] = 'Unknown'

# Additional: add index columns for traceability
feat['raw_index'] = df.index
feat['Timestamp'] = df[find_col(df, "timestamp")] if find_col(df,"timestamp") else np.nan

# One-hot encode occupation and main_avenue (limited cardinality)
for c in ['occupation', 'main_avenue']:
    dummies = pd.get_dummies(feat[c], prefix=c, dummy_na=False)
    feat = pd.concat([feat.drop(columns=[c]), dummies], axis=1)

# Final small housekeeping: fill numeric NaNs with median (conservative)
num_cols = feat.select_dtypes(include=[np.number]).columns.tolist()
feat[num_cols] = feat[num_cols].apply(lambda x: x.fillna(x.median()), axis=0)

# Save processed survey features
out_path = os.path.join(PROC_DIR, "survey_features.csv")
feat.to_csv(out_path, index=False)
print(f"✅ survey_features saved to: {out_path}")
print("Feature columns:", feat.columns.tolist())
feat.head(6)


✅ survey_features saved to: /content/drive/My Drive/portfoliai/data/processed/survey_features.csv
Feature columns: ['gender_male', 'age', 'education_level', 'invests', 'invest_prop_pct', 'monitor_freq_per_month', 'expected_return_pct', 'horizon_years', 'rank_equity', 'rank_mutuals', 'invests_stock', 'raw_index', 'Timestamp', 'occupation_Others', 'occupation_Salaried', 'occupation_Self-employed', 'occupation_Student', 'main_avenue_Bonds', 'main_avenue_Equity', 'main_avenue_Fixed Deposits', 'main_avenue_Gold / SGBs', 'main_avenue_Mutual Funds', 'main_avenue_PPF - Public Provident Fund']


,gender_male,age,education_level,invests,invest_prop_pct,monitor_freq_per_month,expected_return_pct,horizon_years,rank_equity,rank_mutuals,...,occupation_Others,occupation_Salaried,occupation_Self-employed,occupation_Student,main_avenue_Bonds,main_avenue_Equity,main_avenue_Fixed Deposits,main_avenue_Gold / SGBs,main_avenue_Mutual Funds,main_avenue_PPF - Public Provident Fund
0,1.0,39,4,1,0.15,1,0.15,7,3,3,...,False,True,False,False,False,False,True,False,False,False
1,1.0,30,4,1,0.25,4,0.15,7,6,3,...,False,True,False,False,False,True,False,False,False,False
2,0.0,26,4,0,0.40,4,0.15,7,4,7,...,False,True,False,False,False,False,False,False,True,False
3,1.0,32,4,1,0.15,1,0.15,4,5,7,...,False,True,False,False,False,False,False,False,True,False
4,1.0,28,4,1,0.40,30,0.15,2,6,4,...,False,True,False,False,False,True,False,False,False,False
5,1.0,35,4,1,0.25,1,0.15,4,1,3,...,False,True,False,False,False,True,False,False,False,False


You now have a survey_features.csv file with numeric, model-ready features derived from investors.csv. Key features include:

age, gender_male, invest_prop_pct, monitor_freq_per_month, expected_return_pct, horizon_years, rank_equity, rank_mutuals, invests_stock, plus one-hot columns for occupation / main_avenue.

This is the foundation for two parallel approaches:

Survey-only modeling (useful right now) — produce a continuous survey risk score and archetype predictions from these features so new users can be profiled at onboarding.

Merge + supervise (when transaction_personas mapping exists) — later we will try to match these survey records to transaction_personas (if you can supply a customerID or mapping). If a merge is possible, we can train a model that maps survey → transaction-derived labels (preferred because it ground-truths the survey labels to real behavior). If not, we’ll rely on survey-only pseudo-labeling (heuristic + clustering) until you collect transactions from onboarded users.

Next cell: build a survey risk score (transparent heuristic), cluster into 3 archetypes (SC's three), train a small regressor/classifier, and show results. It will also attempt a merge with transaction_personas if possible, and compare.

This next step converts survey_features into:

a survey-derived continuous risk score (0–1) using a simple, explainable formula (combining monitoring frequency, expected return, invest proportion, equity preference, inverse of time horizon).

a 3-cluster segmentation that we will map to the Standard Chartered archetypes (Enthusiastic / Comfortable / Conservative) by cluster mean risk.

a small RandomForest model (regression for continuous risk, classifier for archetype label) to give you a deployable pipeline for onboarding.
Because your survey has ~130 rows, expect noisy performance — this is a prototype to use for onboarding and collecting labels; later we’ll replace/augment with a model trained against transaction_personas once you can join records.

In [6]:
# === Cell 4: Survey risk score, 3-cluster archetypes, and small model training ===

import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold, KFold

PROC_DIR = "/content/drive/My Drive/portfoliai/data/processed"
feat_path = os.path.join(PROC_DIR, "survey_features.csv")
feat = pd.read_csv(feat_path)
print("Loaded survey features:", feat.shape)

# 1) Build a transparent heuristic risk score from survey features
#    Aggressiveness proxy = monitor_freq + expected_return + invest_prop + equity_pref_norm + (1/horizon_years)
#    weights chosen to be interpretable; we'll scale to 0-1 afterwards.

# Safeguard for horizon (avoid divide by zero)
feat['horizon_years'] = feat['horizon_years'].replace(0, np.nan).fillna(feat['horizon_years'].median())
feat['inv_horizon'] = 1.0 / feat['horizon_years']

# equity preference: if rank_equity exists, treat larger rank = more preference; else use invests_stock
if 'rank_equity' in feat.columns and feat['rank_equity'].notnull().any():
    # normalize rank_equity so higher => more preference
    equity_raw = feat['rank_equity'].fillna(feat['rank_equity'].median()).astype(float)
else:
    equity_raw = feat['invests_stock'].astype(float)

# Compose raw score
# adjust weights (you can tune later)
feat['risk_raw'] = (
    0.28 * feat['monitor_freq_per_month'].astype(float) +       # active monitoring -> more aggressive
    0.28 * feat['expected_return_pct'].astype(float).fillna(0) +# expects higher returns -> more aggressive
    0.20 * feat['invest_prop_pct'].astype(float).fillna(0) +    # investing more proportion -> more aggressive
    0.14 * equity_raw.astype(float).fillna(0) +                # preference for equity -> aggressive
    0.10 * feat['inv_horizon'].astype(float).fillna(0)         # shorter horizon -> more aggressive
)

# 2) Scale to 0-1
scaler = MinMaxScaler()
feat['survey_risk_score'] = scaler.fit_transform(feat[['risk_raw']])

print("=== Survey risk score (summary) ===")
print(feat['survey_risk_score'].describe().round(4))

# 3) 3-cluster segmentation (map to SC's three archetypes)
kmeans = KMeans(n_clusters=3, random_state=42, n_init=20)
# use a small set of scaled features for clustering (monitor, return, invest_prop, equity, inv_horizon)
cluster_features = ['monitor_freq_per_month','expected_return_pct','invest_prop_pct','inv_horizon']
cluster_df = feat[cluster_features].fillna(0)
cluster_df = pd.DataFrame(scaler.fit_transform(cluster_df), columns=cluster_df.columns)

kmeans.fit(cluster_df)
feat['cluster'] = kmeans.labels_

# map cluster -> archetype label by average survey_risk_score per cluster
cluster_order = feat.groupby('cluster')['survey_risk_score'].mean().sort_values(ascending=False).index.tolist()
# highest-risk cluster -> Enthusiastic (similar to SC's Enthusiastic)
cluster_map = {
    cluster_order[0]: "Enthusiastic",   # highest mean risk
    cluster_order[1]: "Comfortable",
    cluster_order[2]: "Conservative"
}
feat['survey_archetype'] = feat['cluster'].map(cluster_map)

print("\n=== Archetype counts ===")
print(feat['survey_archetype'].value_counts())

# 4) Train simple models (regressor for continuous score; classifier for archetype)
X = pd.get_dummies(feat.drop(columns=['raw_index','Timestamp','risk_raw','survey_risk_score','cluster','survey_archetype']), drop_first=True)
y_reg = feat['survey_risk_score']
y_clf = feat['survey_archetype']

# If dataset small, cross-validate conservatively
rf_reg = RandomForestRegressor(n_estimators=100, random_state=42)
rf_clf = RandomForestClassifier(n_estimators=200, random_state=42)

# Evaluate regression with 5-fold CV (R^2)
cv = KFold(n_splits=5, shuffle=True, random_state=42)
reg_scores = cross_val_score(rf_reg, X, y_reg, cv=cv, scoring='r2')
print("\nRegression R^2 (5-fold CV):", np.round(reg_scores, 3), "mean:", np.round(reg_scores.mean(),3))

# Evaluate classifier with stratified CV (accuracy)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
clf_scores = cross_val_score(rf_clf, X, y_clf, cv=skf, scoring='accuracy')
print("Classifier accuracy (5-fold CV):", np.round(clf_scores,3), "mean:", np.round(clf_scores.mean(),3))

# Fit final models on all data and save them for deployment
rf_reg.fit(X, y_reg)
rf_clf.fit(X, y_clf)

import joblib
joblib.dump(rf_reg, os.path.join(PROC_DIR, "survey_rf_regressor.pkl"))
joblib.dump(rf_clf, os.path.join(PROC_DIR, "survey_rf_classifier.pkl"))
print("\n✅ Saved survey models to processed folder.")

# 5) Attempt to compare/merge with transaction_personas if available
if transaction_personas is not None:
    # aggregate transaction_personas per customerID (mean risk_score_scaled)
    agg_trans = transaction_personas.groupby('customerID').agg({
        'risk_score_scaled': 'mean',
        'abs_persona': lambda s: s.mode().iat[0] if not s.mode().empty else np.nan,
        'pct_persona': lambda s: s.mode().iat[0] if not s.mode().empty else np.nan
    }).reset_index()
    print("\nTransaction personas aggregated (unique customers):", agg_trans.shape[0])

    # Try to join: look for a matching column in survey (customerID, Username, or other)
    join_key_survey = None
    if 'customerID' in feat.columns:
        join_key_survey = 'customerID'
    elif 'Username' in survey_df.columns and survey_df['Username'].notna().any():
        join_key_survey = 'Username'

    if join_key_survey:
        merged = pd.merge(feat, agg_trans, left_on=join_key_survey, right_on='customerID', how='left')
        print("\nMerged survey -> transaction_personas (rows):", merged.shape[0])
        # show comparison where both exist:
        cmp = merged.dropna(subset=['risk_score_scaled'])
        if not cmp.empty:
            # Correlate survey_risk_score vs transaction risk
            corr = cmp['survey_risk_score'].corr(cmp['risk_score_scaled'])
            print("Correlation (survey_risk_score vs transaction risk_score_scaled):", np.round(corr,3))
        else:
            print("No matching records found when attempting to merge survey <-> transaction_personas.")
    else:
        print("\n⚠️ Could not auto-merge survey and transaction_personas (no shared key found).")
        print("If you have a mapping (survey Username -> customerID), upload it as e.g. 'username_to_customerID.csv' in the processed folder and we will merge in the next cell.")
else:
    print("\nNo transaction_personas available to compare (transaction_personas is None).")

# Save enriched survey file with labels/predictions
feat_out = feat.copy()
feat_out['survey_risk_score'] = feat['survey_risk_score']
feat_out['survey_archetype'] = feat['survey_archetype']
feat_out.to_csv(os.path.join(PROC_DIR, "survey_with_scores_and_archetypes.csv"), index=False)
print("\n✅ survey_with_scores_and_archetypes.csv written.")


Loaded survey features: (132, 23)
=== Survey risk score (summary) ===
count    132.0000
mean       0.2631
std        0.3503
min        0.0000
25%        0.0625
50%        0.0980
75%        0.1738
max        1.0000
Name: survey_risk_score, dtype: float64

=== Archetype counts ===
survey_archetype
Conservative    71
Comfortable     38
Enthusiastic    23
Name: count, dtype: int64

Regression R^2 (5-fold CV): [1. 1. 1. 1. 1.] mean: 1.0
Classifier accuracy (5-fold CV): [0.926 1.    0.962 0.923 1.   ] mean: 0.962

✅ Saved survey models to processed folder.

Transaction personas aggregated (unique customers): 28770

⚠️ Could not auto-merge survey and transaction_personas (no shared key found).
If you have a mapping (survey Username -> customerID), upload it as e.g. 'username_to_customerID.csv' in the processed folder and we will merge in the next cell.

✅ survey_with_scores_and_archetypes.csv written.


At this stage, we now have two distinct sources of persona data:

Survey-based personas

Built from investors.csv, where respondents answered structured questions about demographics, investment objectives, monitoring frequency, and preferred investment avenues.

From this data, we engineered features and trained models that output both a numeric risk score and a categorical archetype (Conservative, Comfortable, Enthusiastic).

This engine is especially useful for new users during onboarding, since they have no transactional history yet.

Transaction-based personas

Derived from transaction_personas.csv, which was aggregated from 32K transactions into ~28K unique customers.

Each customer is assigned a scaled risk score and mapped to persona buckets (Conservative, Balanced, Aggressive, etc.).

This engine is designed for active users, where behavior can reveal actual risk appetite.

⚠️ Key limitation:
The survey respondents are not the same individuals as the transaction dataset customers. The Username in the survey file does not match any customerID in the transaction file. This means we cannot perform a direct merge (row-by-row join).

Instead, the two engines must be treated as parallel and complementary:

The Survey Engine provides initial personas at onboarding.

The Transaction Engine takes over once users start making transactions.

In future work, if overlapping users exist (e.g., survey respondents who also transact), we could build a mapping file (username_to_customerID.csv) to calibrate the two engines against each other.

✅ Summary so far:

Both engines are functional and produce risk scores + personas independently.

No direct merge is possible at this stage, but the outputs are conceptually aligned.

This sets the foundation for a dual-engine architecture in our app:

Survey-driven risk profiling for new users.

Transaction-driven dynamic profiling for active users.

In [7]:
# --- Cell 6: Closing Notebook 7 ---

# Save enriched survey dataset (already done in Cell 4, but confirm path)
survey_out = os.path.join(base_path, "survey_with_scores_and_archetypes.csv")
print(f"✅ Survey personas available at: {survey_out}")

# Transaction personas were already saved in Notebook 6
trans_out = os.path.join(base_path, "transaction_personas.csv")
print(f"✅ Transaction personas available at: {trans_out}")

print("\n📌 Both survey-based and transaction-based persona engines are ready.")
print("⚠️ Remember: they currently operate independently (no shared user IDs).")


✅ Survey personas available at: /content/drive/My Drive/portfoliai/data/processed/survey_with_scores_and_archetypes.csv
✅ Transaction personas available at: /content/drive/My Drive/portfoliai/data/processed/transaction_personas.csv

📌 Both survey-based and transaction-based persona engines are ready.
⚠️ Remember: they currently operate independently (no shared user IDs).


📘 Closing Notebook 7: Dual Persona Engines

At this point, we have completed the Survey Persona Engine:

It reads the survey dataset (investors.csv),

Engineers features related to investor behavior,

Generates both a numeric risk score and a categorical archetype,

Saves the enriched file as survey_with_scores_and_archetypes.csv.

Together with the Transaction Persona Engine from Notebook 6, we now have:

Survey-based personas → useful for onboarding new users.

Transaction-based personas → useful for dynamic updates as users interact with the app.

⚠️ Key Insight:
The two datasets do not overlap in users (no shared IDs). This means we cannot merge them directly at the dataset level. Instead, we treat them as two complementary systems:

Onboarding → Assign persona using Survey Engine.

App usage → Update persona dynamically using Transaction Engine.

✅ Next Steps (Notebook 8):
In the next notebook, we will:

Integrate survey and transaction personas conceptually into a unified framework.

Define business rules for switching from survey-based risk profiling to transaction-based profiling.

Explore how the dual-engine design can be used inside the app (e.g., first-time onboarding vs. ongoing monitoring).

This way, Notebook 8 becomes the place where we connect both engines logically, even if they don’t merge on raw IDs.